#### Environment

 * [Visual Studio Code IDE](https://code.visualstudio.com/?wt.mc_id=DX_841432)
   * [Jupyter Extension for Visual Studio Code](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)
   * [Python v3.9.13](https://www.python.org/downloads/release/python-3913/)

#### Imports

 * [Pandas](https://pandas.pydata.org/)
 * [NumPy](https://numpy.org/)
 * [missingno](https://github.com/ResidentMario/missingno)
 * [matplotlib](https://matplotlib.org/stable/index.html)
 * [sys](https://docs.python.org/3/library/sys.html)
 * [Seaborn](https://seaborn.pydata.org/)

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import sys
import seaborn as sb
from sklearn.decomposition import PCA

#### Load data & Consistant Clean-up

File: medical_raw_data.csv
<br><br>
Description:
    
* 1. Column is an unamed numbered column. We will be skipping this.
* 2. Datetypes will be converted to appropriate values

In [ ]:
print(f"Current Python Version: {sys.version}")

file = './medical_raw_data.csv'

use_columns = range(1, 53)  # Skip first column
medical_data = pd.read_csv(file, header = 0, usecols = use_columns)

convert_to_category = {
    'Zip': 'category',
    'City': 'category',
    'State': 'category',
    'County': 'category',
    'Area': 'category',
    'Timezone': 'category',
    'Job': 'category',
    'Education': 'category',
    'Employment': 'category',
    'Marital': 'category',
    'Gender': 'category',
    'ReAdmis': 'category',
    'Soft_drink': 'category',
    'Initial_admin': 'category',
    'HighBlood': 'category',
    'Stroke': 'category',
    'Complication_risk': 'category',
    'Arthritis': 'category',
    'Diabetes': 'category',
    'Hyperlipidemia': 'category',
    'BackPain': 'category',
    'Allergic_rhinitis': 'category',
    'Reflux_esophagitis': 'category',
    'Asthma': 'category',
    'Services': 'category',
    'Item1': 'category',
    'Item2': 'category',
    'Item3': 'category',
    'Item4': 'category',
    'Item5': 'category',
    'Item6': 'category',
    'Item7': 'category',
    'Item8': 'category'
}

#update datatype listed in convert_to_category to category datatype
medical_data = medical_data.astype(convert_to_category)

#these are categorical variables(column) that need to be converted into Yes/No later for data uniformity
convert_to_category.update({'Overweight': 'category', 'Anxiety': 'category', })


#### Observe the Data Frames Information

In [ ]:
medical_data.info()

#### Detecting Duplicates

###### Duplicate detection methods
* Detecting duplicates as a row across all columns. 
  * Pandas provides `pandas.DataFrame.duplicate()` function. This checks if rows are duplicated across all the columns.
    * [Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html)


In [ ]:
all_columns_duplicates = medical_data.duplicated()
print(all_columns_duplicates.value_counts())

medical_duplicate_rows = medical_data[all_columns_duplicates]

#Assert that we have 0 full row duplicates
assert len(medical_duplicate_rows.values) == 0

duplicate_customer_ids_groups = medical_data.duplicated(subset = ['Customer_id', 'Interaction', 'UID'])
print(duplicate_customer_ids_groups.value_counts())

customer_id_duplicate_rows = medical_data[duplicate_customer_ids_groups]

#Asset that we do not have any of the 3 customer id types as duplicated
assert len(customer_id_duplicate_rows) == 0

duplicate_caseorder = medical_data.duplicated(subset = ['CaseOrder'])
print(duplicate_caseorder.value_counts())

caseorder_duplicates = medical_data[duplicate_caseorder]

#Assert that we do not have any duplicated Case Orders
assert len(caseorder_duplicates) == 0

#### Creating Functions To Assist In Missing Data Detection:

##### Description:
    Becaues each variable (column) get's checked for missing values with the same Data Frame functions [`isna()`, `sum()`] and steps.
    There are two functions:
    
  * **Function:** `checkForMissingValues(series, columnName)`
    * This takes a Data Frame series and a specific column and runs Pandas isna() for evaluating the missing values.
    * Pandas sum() is then used to get the count of missing data. If that count exceeds 0 messages will be printed to inform that missing values were found and the count returned.
  <br>
  * **Function:** `checkForMissingValuesByColumns(series, columns)`
    * This takes a Data Frame series and a list of column names to check for missing values.
    * This function will loop through the list of columns and call `checkForMissingValues` for the result
      * After all the columns have been evaluated for missing values a new Data Frame is created that holds Column Name and # of missing values for that column

In [ ]:
#Creating helper functions for detection of missing data

def checkForMissingValues(series: pd.Series, columnName: str):
    naCount = 0
    try:
        naInfo = series[columnName].isna()
        naCount = naInfo.sum()
        #print(f'{columnName} total missing count: {naCount}\n')

        assert naCount == 0
    except AssertionError:
        print(f'Missing data detected for {columnName} column of Data Frame')
    except:
        print(f'Unknown column found {columnName}')
    return naCount

def checkForMissingValuesByColumns(series, columns=[]):
    naCounts = []
    for column in columns:
        naCount = checkForMissingValues(series, column)
        naCounts.append(naCount)

    missingValuesResult = {
        'Column': columns,
        'NA_Count': naCounts
    }

    naDataFrame = pd.DataFrame(missingValuesResult)

    return naDataFrame


#### Detecting Missing Values

##### Description:

We will be using the resulting Data Frame naReport that was generated by our custom function that checked all the columns for missing data. We will retrieve any observation (row) that includes `NA_Count > 0` and store this is a `missing_data` Data Frame to use for the visualization verification of missing data.

In [ ]:
print('\nChecking for missing data in each column. \n')

naReport = checkForMissingValuesByColumns(medical_data, medical_data.columns)
missing_data = naReport[naReport['NA_Count'] > 0]
missing_data

#### Visualizing Missing Data

##### Description:

After gathering which columns contain NA/NULL values into Series and will be using the missingno & matplotlib library combo. library to generate a chart that visualizes the gaps within the data. The whitespace within the histogram chart represents missing values.

In [ ]:
missing_data_columns = missing_data['Column']

msno.matrix(medical_data[missing_data_columns], fontsize = 12, labels = True)
plt.title('Columns with Missing Data')
plt.show()

#Need to filter out categorical columnas as those are a MODE Imputation. Need to check numerical columns for Normal or Left/Right Skew.
category_columns = pd.Series(list(convert_to_category.keys()))
numerical_nas = missing_data_columns[~missing_data_columns.isin(category_columns)]

for numerical_column in numerical_nas:
    df = medical_data[numerical_column]
    df.hist()
    plt.title(f"Distribution of '{numerical_column}'")
    plt.show()


#### Detecting Outliers

##### Description:

Here we will gather the columns that have a numerical data type of `int64` and `float64`. We'll loop through each column name and create and display a boxplot. Each boxplot will need to be visually checke to see if there are any data points out side of the boxplot whiskers, which indicates there are outliers.

In [ ]:
# Interquartile Range (IQR)
# Outliers are values falling outside the 25th and 75th percentile
# https://www.vedantu.com/maths/interquartile-range
def getOutliers_IQR(dataframe: pd.DataFrame, columnName: str):
    variableOfInterest = dataframe[columnName]

    quantile1 = variableOfInterest.quantile(0.25)
    quantile3 = variableOfInterest.quantile(0.75)

    interquartile_range = quantile3 - quantile1

    below_25_select = variableOfInterest < (quantile1 - 1.5 * interquartile_range)
    above_75_select = variableOfInterest > (quantile3 + 1.5 * interquartile_range)

    outliers = variableOfInterest[below_25_select | above_75_select]

    summary = {
        'Variable': [columnName],
        'Total_Outliers': [outliers.count()],
        'Outlier_Min_Value': [outliers.min()],
        'Outlier_Max_Value': [outliers.max()]
    }

    summary_dataframe = pd.DataFrame(summary)

    summary_dataframe.fillna(value=0, inplace=True)

    return summary_dataframe

# Get all numerical columns
numerical_columns = medical_data.select_dtypes(include=[np.int64, np.float64]).columns

numerical_quantitative = numerical_columns.get_indexer_for(['CaseOrder', 'Overweight', 'Anxiety'])
numerical_columns = numerical_columns.delete(numerical_quantitative)

print(f"Boxplots to be generated '{numerical_columns.size}'")

for column in numerical_columns:
    sb.boxplot(x = medical_data[column])
    plt.title(f"Boxplot of {column}")
    plt.show()
    print(getOutliers_IQR(medical_data, column))


#### Treatment for Duplicates

##### Description:

What defined a duplicate did not produce any entries. There are no treatments needed for duplicates for the `Medical Data Set`

#### Treatments for Missing Data

##### Description:
    The columns listed below with missing data we will chart their distribution to help choose with statistical central tendency value to use for imputation.

**Columns with missing data**
* Children - 2588 - Numerical
* Age -2414 - Numerical
* Income - 2464 - Numerical
* Soft_drink - 2467 - Categorical
* Overweight - 982 - Categorical (Numerical needing converting to Categorical Yes/No)
* Anxiety - 984 - Categorical (Numerical needing converting to Categorical Yes/No)
* Initial_days - 1056 - Numerical

<br>



In [ ]:
# ‘kde’ : Kernel Density Estimation plot
  
imputations = {
     'Children': int(medical_data['Children'].median()) #Right skewed
    ,'Age': int(medical_data['Age'].mode()) #U-Shaped [Bi-Modal]
    ,'Income': medical_data['Income'].median() #Right skewed
    ,'Soft_drink': medical_data['Soft_drink'].mode()[0] #Categorical
    ,'Overweight': int(medical_data['Overweight'].mode()[0]) #Categorical
    ,'Anxiety': int(medical_data['Anxiety'].mode()[0]) #Categorical
    ,'Initial_days': medical_data['Initial_days'].mode()[0] #U-Shaped Bi-Modal
}

imputated_medical_data = medical_data.fillna(imputations)

#Visualization check
msno.matrix(imputated_medical_data[missing_data_columns], fontsize=12, labels=True)
plt.title('Missing Data columns after imputation.')
plt.show()

for numerical_column in numerical_nas:
    numericals = medical_data[numerical_column]
    numericals.hist()
    plt.title(f"Distribution of '{numerical_column}' after imputation.")
    plt.show()

convert_to_int64 = {
    'Age': np.int64,
    'Children': np.int64
}

imputated_medical_data = imputated_medical_data.astype(convert_to_int64)

#### Treatment for Outliers

##### Description:

The treatment plan for the outliers is in the `Medical Dataset` is Retain with noting of their existence. The values that have presented themselves as outliers are valid values within the context of the given variables. There is one variable (column) `VitD_levels` that have a weird mix of outliers at first glance but are valid. To help with the understanding of the `VitD_levels` a new variable(column) will be calculated. It will be the human readable version of the variables that falls into 3 levels/bins. Low, Adequate, and High are categorical values that correspond to Vitmin D Level ranges.

**Vitamin D Level Ranges:**
* Low -> 12 or below
* Adequate -> 20 or above
* High -> 50 or above 

In [ ]:
# https://www.medicalnewstoday.com/articles/normal-vitamin-d-levels#function
vitamin_d_bins = [0, 20, 50, np.inf]
vitamin_d_categories = ['Low', 'Adequate', 'High']

vitd_levels_group = pd.cut(imputated_medical_data['VitD_levels'], bins=vitamin_d_bins, labels=vitamin_d_categories)

vitd_level_index = imputated_medical_data.columns.get_loc('VitD_levels')
imputated_medical_data.insert((vitd_level_index + 1), 'VitD_levels_level', vitd_levels_group)

chart_data = imputated_medical_data['VitD_levels_level'].value_counts()
chart_data.plot.bar(x='Vitamin D Range', y='Customer Count')
plt.title('Number of customers in each Vitamin D range.')
plt.show()


#### Detecting 0/1's that need Re-Expression

In [ ]:
noyes_as_bits = [0, 1]
columns_to_reexpress = []

numerical_columns = medical_data.select_dtypes(include=[np.int64, np.float64]).columns

for column in numerical_columns:
    if imputated_medical_data[column].isin(noyes_as_bits).all():
        columns_to_reexpress.append(column)

print(f"Re-Expression needed for {columns_to_reexpress}")


#### Treatment for Re-Expression


In [ ]:
# Detected values of pure 0/1's should be Re-Expressed into Yes/No.
for column in columns_to_reexpress:
    imputated_medical_data[column] = imputated_medical_data[column].map({1: 'Yes', 0: 'No'})

imputated_medical_data[columns_to_reexpress].head()

#### Exporting the finalized cleaned-up data to csv file

##### Description:

* [to_csv documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html)

In [ ]:
imputated_medical_data.to_csv(r'./medical_cleaned_data.csv')

#### Principle Component Analysis (PCA)

* Must be run on cleaned data
* Must only include numerical data of the same scale
* Group each component based on similarity

In [ ]:
medical_data_clean = imputated_medical_data

#Use Discrete and Continious data only
pca_columns = ['Lat', 'Lng', 'Population', 'Age', 'Children', 'Income', 'VitD_levels', 'Full_meals_eaten', 'VitD_supp', 'Initial_days', 'TotalCharge', 'Additional_charges']

pca_df = medical_data_clean[pca_columns]

#normalization (AH HA MOMENT: This makes sense as to why outliers can be damaging to analysis)
#Wildly skewed outliers will pull the math in uncertain directions
pca_normalized = (pca_df - pca_df.mean()) / pca_df.std()

pca = PCA(n_components=pca_normalized.shape[1])
print(pca) #Should be PCA(n_component=12)

pca.fit(pca_normalized)


component_columns = []
for i in range(1, len(pca_columns) + 1):
    component_columns.append(f"PC{i}")

pca_result = pd.DataFrame(pca.transform(pca_normalized), columns=component_columns)
#pca_result

pca_loadings = pd.DataFrame(pca.components_.T, columns=component_columns, index=pca_normalized.columns)
pca_loadings

#### Selecting PCs

##### Kaiser Rule (Eigenvalues):
Must calculate covariance and vectors; then define eigenvalues before performing scree plot

In [ ]:
covariant_matrix = np.dot(pca_normalized.T, pca_normalized) / pca_df.shape[0]
eigenvalues = [np.dot(eigenvector.T, np.dot(covariant_matrix, eigenvector)) for eigenvector in pca.components_]

print(eigenvalues)

plt.plot(eigenvalues)
plt.xlabel('Component Number')
plt.ylabel('Eigenvalues')
plt.axhline(y=1, color='red')
plt.show()
